In [1]:
from color_palette import PaletteCreator

import cv2
import numpy as np
import pandas as pd
import os

In [2]:
pc = PaletteCreator()

In [3]:
data_root = './images/dataset'
seasons = ['spring', 'summer', 'fall', 'winter']

In [4]:
def bgr2cmyk(bgr):
    b, g, r = bgr
    c = 1 - r / 255
    m = 1 - g / 255
    y = 1 - b / 255
    k = min(c, m, y)
    
    c = (c - k) / (1 - k)
    m = (m - k) / (1 - k)
    y = (y - k) / (1 - k)
    
    return np.array([c, m, y, k]).swapaxes(0, 2)

In [5]:
df = pd.DataFrame(columns=['H', 'S', 'V', 'L', 'A', 'B', 'C', 'M', 'Y', 'K', 'label'])

In [6]:
for folder in seasons:
    this_df = pd.DataFrame(columns=['H', 'S', 'V', 'L', 'A', 'B', 'C', 'M', 'Y', 'K', 'label'])
    
    for file in os.listdir(os.path.join(data_root, folder)):
        full_path = os.path.join(data_root, folder, file)
        
        palette = pc.create_palette(full_path)
        if palette is None:
            continue
        palette = np.array([palette], np.uint8)
        
        hsv_palette = cv2.cvtColor(palette, cv2.COLOR_BGR2HSV)
        lab_palette = cv2.cvtColor(palette, cv2.COLOR_BGR2LAB)
        cmyk_palette = bgr2cmyk(palette)
        
        mean_hsv = np.mean(hsv_palette, axis=1)
        mean_lab = np.mean(lab_palette, axis=1)
        mean_cmyk = np.mean(cmyk_palette, axis=1)
        
        row = np.concatenate((mean_hsv, mean_lab, mean_cmyk), axis=1).tolist()
        row.append(folder)
        this_df[len(this_df)] = row
        
    this_df.to_csv(f'images/dataset/mean_{folder}.csv', index=False)
        
    pd.concat([df, this_df], axis=0)

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
df.to_csv('images/dataset/mean.csv', index=False)

In [ ]:
print(f'Finished creating dataset for {len(df)} images')